## Introduction to VectorCube, CubeView, and SMAdapter

___


This Notebook demonstrates the basics of the [VectorCube](../rubiks/model/VectorCube.py) class and displaying it with the [CubeView](../rubiks/model/CubeView.py) class, both in 2D projection or in 3D. This includes the use of color and alpha mask options in displays and my example using this feature in 3D to observe the tendency (in contrast to human solutions) of effiecient solution to be unconcerned with returning individual to their solved positions until the last minute.

Note that these cells employ both the use of local-move notation [e.g. **(WHITE_CB, -90), (BLUE_CB, 180)**] as well as Singmaster notation [e.g. **"U'", "L2"**] using the [SMAdapter](../rubiks/model/SMAdapter.py) (which functions essentially like a wrapper class for the cube), and how you can easily mix these notations as well. For reference, the full set of SMAdapter supported Singmaster notation is:


| Move Type | Singmaster Notation |
| :------ | :----------- |
| Clockwise face rotations:   | **[U, L, F, R, B, D, U2, L2, F2, R2, B2, D2]** |
| Counterclockwise face:      | **[U', L', F', R', B', D']** |
| Slice turns:                | **[M, M', E, E', S, S', M2, E2, S2]** |
| Double layer turns:         | **[u, l, f, r, b, d, u2, l2, f2, r2, b2, d2]** |
| Inverse double layer turns: | **[u', l', f', r', b', d']** |
| Whole cube rotations:       | **[X, X', Y, Y', Z, Z', X2, Y2, Z2]** |

___


In [ ]:
import sys

# This for managing relative imports from nb
if '..' not in sys.path: sys.path.append('..')

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np

from rubiks.model.CubeView import CubeView
from rubiks.model.SMAdapter import SMAdapter
from rubiks.model.DirectCube import DirectCube
from rubiks.model.VectorCube import VectorCube, SIDES, color_name, color_letr

from rubiks.solver.CyclicSolver import CyclicSolver
from rubiks.solver.FaceletSolver import FaceletSolver
from rubiks.solver.DirectSolver import DirectSolver

In [ ]:
# Simple demo of basic Vector representation of a 
# Rubik's Cube (scrambling/rotating sides) with the 
# CubeView class displaying moves w/2D-projections

# Projection display
cube = VectorCube()
view = CubeView(cube)
moves, invmoves = cube.trace_scramble()
before_state = cube.state()

print("SCRAMBLING:\n")
for mv in moves:
    cube.rotate(mv)
    view.push_snapshot(caption=f"{color_name(mv[0])} ({mv[1]})")
    
view.draw_snapshots()
view.reset_snapshots()

print("\nREVERSING SCRAMBLE:\n")
for inv in invmoves:
    cube.rotate(inv)
    view.push_snapshot(caption=f"{color_name(inv[0])} ({inv[1]})")

view.draw_snapshots()
print("\nBefore/after state match:", sum(before_state == cube.state()) == len(before_state))

In [ ]:
# Little demo employing the use of an index mask with CubeView 2D-projection display
def get_index(colors, cis): return np.array([i for side, ci in zip(colors, cis) for i, flet
                                             in enumerate(VectorCube._facelet_matrix.T) 
                                             if (flet[0] == side) and (flet[1] == ci)])

corn_idx = [i for sd in SIDES for i in get_index(np.broadcast_to([sd], (16,)), [0,2,8,6])]
edge_idx = [i for sd in SIDES for i in get_index(np.broadcast_to([sd], (16,)), [1,5,7,3])]
corn_idx_shft = [i for sd in SIDES for i in get_index(np.broadcast_to([sd], (16,)), [2,8,6,0])]

ring = []
view = CubeView(VectorCube())
for i, bar in enumerate(np.array([corn_idx, edge_idx, corn_idx_shft]).T):
    ring.extend(bar)
    view.push_snapshot(flet_idx=np.concatenate((bar, VectorCube._centers)))
    if ((i+1) % 4) == 0:
        view.push_snapshot(flet_idx=ring)
        view.draw_snapshots()
        view = CubeView(VectorCube())
        ring = []

In [ ]:
# Similarly a simple test of basic CfopCube representation of a 
# Rubik's Cube (scrambling/rotating sides), this time with the 
# CubeView class displaying an animated 3D cube image

# 3D display
inner_cube = VectorCube()
cube_adpt = SMAdapter(inner_cube)
view = CubeView(cube_adpt)
view.record_moves()

# Singmaster notation
sm_names, sm_inv_names = cube_adpt.trace_scramble(sz=20)

local_moves = []
for name in sm_names: cube_adpt.rotate_singmaster(name, local_moves)
for iname in sm_inv_names: cube_adpt.rotate_singmaster(iname)
view.record_moves(stop_recording=True)

print("Global moves:", sm_names)
print("Local moves:", [f'({color_letr(mv[0])}:{mv[1]})' for mv in local_moves])

# Display animation
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
# Same idea as previous cell, but only animating solution
# sequence and adding an alpha channel mask such that only
# facelets that are in their solved position are displayed

# 3D display
inner_cube = VectorCube()
cube_adpt = SMAdapter(inner_cube)
view = CubeView(cube_adpt)
local_moves = []

# Singmaster notation
sm_names, sm_inv_names = cube_adpt.trace_scramble(sz=20)
for name in sm_names: cube_adpt.rotate_singmaster(name)
    
# This just for mask/alpha drawing purposes to "finish" drawing all facelets
if sm_names[0] not in SMAdapter._cube_rotations: sm_inv_names += ['Y', "Y'"]

view.record_moves()
for iname in sm_inv_names:
    nhome_mask = np.nonzero(sum(inner_cube.facelet_matrix[2:] != VectorCube._facelet_matrix[2:]) > 0)[0]
    cube_adpt.rotate_singmaster(iname, local_moves=local_moves, alpha_masks=nhome_mask)

view.record_moves(stop_recording=True)

print("Global scramble moves:", sm_names)
print("Local solution moves:", [f'({color_letr(mv[0])}:{mv[1]})' for mv in local_moves])

# Display animation
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
# Same idea as previous cell, but using a local-move that is
# guaranteed to be non-redundant. Often better demonstrates the 
# non-human-solution-like, "entropy" quality of efficient solutions.

# 3D display
inner_cube = VectorCube()
cube_adpt = SMAdapter(inner_cube)
view = CubeView(cube_adpt)

# Singmaster notation
moves, invmoves = inner_cube.trace_scramble(sz=20)
cube_adpt.rotate_local_seq(moves)

view.record_moves()
for invmv in invmoves:
    nhome_mask = np.nonzero(sum(inner_cube.facelet_matrix[2:] != VectorCube._facelet_matrix[2:]) > 0)[0]
    cube_adpt.rotate_local(invmv, alpha_masks=nhome_mask)

for sm_name in ['Y', "Y'"]:
    # Just a full-cube rotation to finish drawing facelets (since my nhome_mask alpha-mask lags one move)
    nhome_mask = np.nonzero(sum(inner_cube.facelet_matrix[2:] != VectorCube._facelet_matrix[2:]) > 0)[0]
    cube_adpt.rotate_singmaster(iname, alpha_masks=nhome_mask)
    
view.record_moves(stop_recording=True)

print("Local solution moves:", [f'({color_letr(mv[0])}:{mv[1]})' for mv in invmoves])

# Display animation
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
# This cell just animates the solution to the "Super Flip" position.
# In this position, all corner facelets are solved, and every edge
# cubelet is in the correct position, but flipped. Solving this 
# permutation requires a minimum 20 (or "God's number") of moves.

sm_superflip_solution = [SMAdapter.inverse(sm_name) for sm_name in reversed(SMAdapter.SM_SUPERFLIP_SCRAMBLE)]
superflip_solution = []

inner_cube = VectorCube()
cube_adpt = SMAdapter(inner_cube)
view = CubeView(cube_adpt)

# Apply a superflip scramble to the cube
cube_adpt.rotate_singmaster_seq(SMAdapter.SM_SUPERFLIP_SCRAMBLE)
# for name in SMAdapter.SM_SUPERFLIP_SCRAMBLE: cube_adpt.rotate_singmaster(name)

# Apply solution
view.record_moves()
cube_adpt.rotate_singmaster_seq(sm_superflip_solution, local_moves=superflip_solution)
# for iname in sm_superflip_solution: cube_adpt.rotate_singmaster(iname, local_moves=superflip_solution)
view.record_moves(stop_recording=True)

print("Superflip solution:", [f'({color_letr(mv[0])}:{mv[1]})' for mv in superflip_solution])

# Display animation
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

## Examination of Heuristic Functions
___


The remainder of this Notebook examine the "Minimum-Distance-Home", "Entropy/Order", and "Dance" heuristics by calculating them across samples of randomly scrambled cubes and graphing resulting average values, standard deviations, and coefficients of variation metrics. In particular, the last metric (and thus heuristic stability across cube permutations) can be seen to significantly improve with "Entropy/Order", and "Dance" heuristics.

___


In [ ]:
csolver = CyclicSolver()
fsolver = FaceletSolver()
sequence_hash = fsolver.create_sequence_hash(wildcard=False)
heuristic_hash = fsolver.create_heuristic_hash(sequence_hash)

# Uncomment one or both of these to view entire sequence_hash (1152 lines)
# fsolver.print_sequence_hash(sequence_hash, exclusive=True)
# fsolver.print_sequence_hash(sequence_hash, exclusive=False)

In [ ]:
# The following three cells examine the three heuristic functions, for cubes
# randomly scrambled out to X rotations (for X across a range of [0,24]). The
# three functions are: the "Minimum-Distance-Home", "Entropy/Order", and the 
# "Dance" heuristic. Graphs (scaled such that values fall in the approx. range 
# of [0,7]) are generated to display average value, standard deviation, and
# the coefficient of variation for each function.

# Increase NUMB_SAMPLES for smoother graphs,
# decrease it to shorten graphing execution time
NUMB_SAMPLES = 5000


# "Minimum-Distance-Home"
values = [[(162 - fsolver.heuristic(VectorCube().scramble(sz=sz))) / 30  for i in range(NUMB_SAMPLES)] for sz in range(25)]
averages = np.average(values, axis=1)
std_devs = np.std(values, axis=1)

plt.figure(2)
plt.plot(averages)
plt.title(f'"Min-Dist-Home" Heuristic Average Val')
plt.ylabel('Value')
plt.xlabel('Number of Scramble Moves')

plt.figure(3)
plt.plot(std_devs)
plt.title(f'"Min-Dist-Home" Heuristic Standard Dev')
plt.ylabel('Standard Deviation')
plt.xlabel('Number of Scramble Moves')

plt.figure(4)
plt.plot((std_devs / averages) * 100)
plt.title(f'"Min-Dist-Home" Coefficient of Variation')
plt.ylabel('Percent Variation')
plt.xlabel('Number of Scramble Moves')

print(f"Heuristic value range: [{np.amin(values)}, {np.amax(values)}]")
print(f"Average heruistic value: [{np.amin(averages)}, {np.amax(averages)}]")

In [ ]:
# "Entropy/Order"
values = [[DirectSolver.order_heuristic(VectorCube().scramble(sz=sz)) / 10.0 for i in range(NUMB_SAMPLES)] for sz in range(25)]
averages = np.average(values, axis=1)
std_devs = np.std(values, axis=1)

plt.figure(2)
plt.plot(averages)
plt.title(f'"Entropy/Ordered" Heuristic Average Val')
plt.ylabel('Value')
plt.xlabel('Number of Scramble Moves')

plt.figure(3)
plt.plot(std_devs)
plt.title(f'"Entropy/Order" Heuristic Standard Dev')
plt.ylabel('Standard Deviation')
plt.xlabel('Number of Scramble Moves')

plt.figure(4)
plt.plot((std_devs / averages) * 100)
plt.title(f'"Entropy/Order" Coefficient of Variation')
plt.ylabel('Percent Variation')
plt.xlabel('Number of Scramble Moves')

print(f"Heuristic value range: [{np.amin(values)}, {np.amax(values)}]")
print(f"Average heruistic value: [{np.amin(averages)}, {np.amax(averages)}]")

In [ ]:
# "Dance" heuristic
values = [[DirectSolver.generate_hstate(DirectCube().scramble(sz=sz)).value / 30.0 for i in range(NUMB_SAMPLES)] for sz in range(25)]
averages = np.average(values, axis=1)
std_devs = np.std(values, axis=1)

plt.figure(2)
plt.plot(averages)
plt.title(f'"Dance" Heuristic Average Val')
plt.ylabel('Value')
plt.xlabel('Number of Scramble Moves')

plt.figure(3)
plt.plot(std_devs)
plt.title(f'"Dance" Heuristic Standard Dev')
plt.ylabel('Standard Deviation')
plt.xlabel('Number of Scramble Moves')

plt.figure(4)
plt.plot((std_devs / averages) * 100)
plt.title(f'"Dance" Heuristic Coefficient of Variation')
plt.ylabel('Percent Variation')
plt.xlabel('Number of Scramble Moves')

print(f"Heuristic value range: [{np.amin(values)}, {np.amax(values)}]")
print(f"Average heruistic value: [{np.amin(averages)}, {np.amax(averages)}]")